In [1]:
import joblib
import sys
import os
alamat = "/".join(os.getcwd().split("\\")[:-2])
sys.path.append(alamat+'/praproses')
import ngrams as ng
from normalisasi import normalisasi as nrm
import token as tkn
import stemming as stm
import cleansing
import stemming
import seleksi_kata as sk
import pandas as pd
import isDigit as isd

In [2]:
# tdidf = joblib.load("model/tdidf.w")
naive_bayes = joblib.load("model/naive_bayes.m")

In [3]:
def praproses(twt, n=6):
    twt = sk.seleksi(twt)
    twt = cleansing.cleaning(twt)
    twt = isd.remove_digit(twt)
    twt = nrm.slangword(twt)
    twt = stm.stemming(twt)
    twt = stm.StopWordRemover(twt)
    twt = ng.ngramku(twt, n=n)['string']
    return twt

In [4]:
praproses("RT @hpsetyawan: @afiadi @Gerindra @prabowo setuju bgt", n=6)

'tuju banget tujubanget'

In [5]:
kalimat = "RT @hpsetyawan: @afiadi @Gerindra @prabowo setuju bgt"
# print(praproses(kalimat))
naive_bayes.predict([praproses(kalimat)])[0]
#memprediksi kalimat di preproses, kemudian di transform menjadi vector,diprediksi menggunakan NB

'positive'

# Prediksi data uji [Merdeka Belajar]

In [6]:
df = pd.read_excel("../../data_uji/PPDB ZONASI.xlsx")
twet_uji = df.Twit.tolist()
label_asli = df.label.tolist()
twet_uji_bersih = list()

for ix, t in enumerate(twet_uji):
    twet_uji_bersih.append(praproses(t))
#     print(ix)
prediksi = naive_bayes.predict(twet_uji_bersih)

from sklearn.metrics import classification_report
y_true = label_asli
y_pred = prediksi
target_names = ['negative', 'positive',]
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    negative       0.60      0.72      0.66       181
    positive       0.72      0.60      0.66       219

    accuracy                           0.66       400
   macro avg       0.66      0.66      0.66       400
weighted avg       0.67      0.66      0.66       400



In [7]:
ix = 0
for twet, asli, pred in zip(twet_uji, label_asli,prediksi):
    if pred!= asli:
        print(ix)
        print("Prediksi:",pred)
        print("asli    :",asli)
        print("    ",twet)
        print("="*100)
    ix+=1

0
Prediksi: negative
asli    : positive
     Yuup... 
Setuju sekali dgn argumen ini... 
Sy merasakan bagaimana tersiksanya anak sy hrs Mati2an belajar salah satu mata pelajaran UN yg tdk disukai hanya utk mengejar nilai bagus secara instan dgn les atau bimbel.
Krn nilai UN ibarat sebab setitik nila,  rusak susu sebelangga
1
Prediksi: negative
asli    : positive
     Yg penting skripsi jangan pak wkwk  biar setrong
2
Prediksi: negative
asli    : positive
     Ujian Standart Nasional itu ada di semua negara maju cuma dgn format yg berbeda, jika mau masuk perguruan tinggi bisa dgn 2 jln, perekrutan berdasarkan minat n bakat n kemampuan atau ujian terbuka bersama..Cc. @jokowi n @Kemdikbud_RI ...
3
Prediksi: negative
asli    : positive
     UN cuma buang2 anggaran, UN bikin anak kelas 1-5, 7-8, 10-11 ke gangggu belajar karena banyak libur di semester 2 akibat sekolah fokus ngurusin kelas 6,9 dan 12. Padahal bayaran nya mah tetep 12bulan. UN juga cuma jadi ajang promosi pemgusaha bimbel.
4
P